# Debugging Notebook

This notebook is for a remote debugging setup. It uses pykx q magic cells to interact with a remote q process (in this case a managed cluster).

## PyKX Reference
[PyKX Jupyter q Magic Command](https://code.kx.com/pykx/2.5/getting-started/q_magic_command.html)   

## Tailing logs
The AWS CLI has the ability to tail cloudwatch logs, you can use the CLI and a terminal window (even in Jupyter) to tail the server logs.

[AWS CLI Reference](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/logs/tail.html)

## Notes
**Debug using a single node cluster**   
  - each cell connects to the remote, a multi-node cluster could connect to other nodes of cluster for each cell (%%q magic) used. Any state (like variables and function definitions) are to a specific node of a cluster, so use only one node. 
    


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from env import *
from managed_kx import *

# Cluster names and database
from config import *

# Specific cluster to connect to
CLUSTER_NAME = RDB_CLUSTER_NAME

# set pykx local q console width and height
kx.q.system.display_size = [50, 1000]


In [2]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = get_client(session=session)

# Tail Log
In a terminal, use the AWS CLI to tail a cluster's log. You can use the Jupyter terminal and arrange the terminal tab below this notebook in jupyter lab.

## Example
**ENV_ID:** Finspace environment id   
**CLUSTER_NAME:** FinSpace cluster name   
```
$ aws logs tail /aws/vendedlogs/finspace/ENV_ID/CLUSTER_NAME --follow
```


In [3]:
# Copy/paste for this cluster
print( f"aws logs tail /aws/vendedlogs/finspace/{ENV_ID}/{CLUSTER_NAME} --follow" )

aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/RDB_basictickdb --follow


# Connect to Cluster
Use components of the connection string and the pykx included q magic to connect to a cluster and interact with it, useful for remote debugging.

**NOTE** You need to permissions to the specific environment and cluster to get it's connection string.


In [4]:
# get the full connection string
conn_str = get_kx_connection_string(client, 
                                  environmentId=ENV_ID, clusterName=CLUSTER_NAME, 
                                   userName=KDB_USERNAME, boto_session=session)

host, port, username, password = parse_connection_string(conn_str)

In [5]:
%%q --host $host --port $port --user $username --pass $password

/ list namespaces
key `
""

/ list contents of a specific namespace
key `.aws

`q`Q`h`j`o`comkxic`kurl`com_kx_log`s`aws`awscust`aws_metrics`rdb`conn`query`u`debug

``args`akda`akre`akif`akp`akm`akv`akdb`akdbs`akeah`aksadb`akcp`akcsp`akscp`akdbp`create_changeset`get_changeset`get_latest_sym_file`copy_database_files`commit_kx_database`get_kx_dataview`update_kx_dataview`sgmtcfg`sgmtcfgs`load_kx_database`_process_clusters_list`list_kx_clusters`_process_node_list`list_kx_cluster_nodes`update_kx_cluster_databases`get_kx_cluster`delete_kx_cluster`stop_current_kx_cluster_creation`cache`db`sdbs`sdep`init_pubsub`del`sel`pub`add`sub`internal_log_info`internal_log_err`run_system_command`s3`logger_info`logger_err`fa_user`apply_auth`get_kx_connection_string`get_kx_node_connection_string`metrics`wraplabels`wrapstring`newmetric`addmetric`updval`on_metrics_poll`on_po`on_pc`on_wo`on_wc`before_pg`after_pg`before_ps`after_ps`before_ph`after_ph`before_pp`after_pp`before_ws`after_ws`before_ts`after_ts`po`pc`wo`wc`pg`ps`ph`pp`ws`ts`overloadhandler`init_metrics`infokeys`infovals`info`me

In [6]:
%%q --host $host --port $port --user $username --pass $password
/ call function on remote cluster
dview:.aws.get_kx_dataview[.rdb.database;.rdb.dbView];

/ display results
dview
""
/ specific elements of results
dview`active_versions
""

dview`segment_configurations

dataview_name          | "basictickdb_DBVIEW"
database_name          | "basictickdb"
status                 | "ACTIVE"
changeset_id           | "pslyVBfwdS41EHpCO31Qqw"
segment_configurations | +`db_paths`volume_name!(,,"/*";,"RDB_TP_SHARED")
availability_zone_id   | "use1-az6"
az_mode                | "SINGLE"
description            | "Dataview of database"
auto_update            | 0b
read_write             | 0b
active_versions        | +`changeset_id`segment_configurations`attached_clusters`created_timestamp`version_id!(,"pslyVBfwdS41EHpCO31Qqw";,+`db_paths`volume_name!(,,"/*";,"RDB_TP_SHARED");,,"HDB_basictickdb";,1.730414e+09;,"LslyWuJFjEE93ZVmHlujFw")
create_timestamp       | 1.730414e+09
last_modified_timestamp| 1.730414e+09

changeset_id             segment_configurations                           attached_clusters created_timestamp version_id              
---------------------------------------------------------------------------------------------------------------------------

In [7]:
%%q --host $host --port $port --user $username --pass $password

/ define a new function on the server 
/ this would likely be a function being developed/tested

.foo.myfunc:{[name]
    nodes: .aws.list_kx_cluster_nodes[name];

    {.aws.get_kx_node_connection_string[x;y]}[name] each nodes`node_id
 }


In [8]:
%%q --host $host --port $port --user $username --pass $password

/ call the function
.foo.myfunc["TP_basictickdb"]

":tcps://ip-192-168-4-150.ec2.internal:443:RDB_basictickdb:Host=ip-192-168-4-150.ec2.internal&Port=443&User=RDB_basictickdb&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjECAaCXVzLWVhc3QtMSJHMEUCIQDZWcXmi%2BOeK9JOyIFqes02mmRtlok7EjFXKIail5F5awIgI%2B1S6LgC1S0SRI2iipIHEvKlVvyeZjqUzcU7zpuaBkUqgQMImf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw4Mjk4NDU5OTg4ODkiDIuHIIE3xlrL%2BqIxWirVAn5JWI30CTUtqC44Uv48tlGKVuWKEBkRKxcR6zx2ryJZYpWgxw%2FFldq7TE2KUpBFNqcqr%2FeFjKvQmxCMKEuwhJk8xzZ%2F%2FnUHBewnhXVpZyV5wTmTgX%2BLQ0hgpOPwfOGCOMdFSdgLRuwySOwGmyARoxwNlCEOzdiRxo7ubR2erExpC%2FngEmAGIyJi9CLTJksyZ7FHy%2BAf9%2BRWrS7CrXLe6hgU6DwenLxrLnMWCdKUvDEdViokVwryUiRA%2BBq%2FIv4%2FoJfWzihVykxCU7sxABDKouOPwPT5JXWkJfWliUZ4mmyEKNi4K2TVi6eqNNuvoz1JtHboa%2FKoncRQmPiqHe3iUtf%2FgD49NmBVzn7h%2FxKjGvN7deg1eUZ1tVjhk5RcZiz6%2Bys34UrHJVOpdHa%2BYyPZcX2wVxkz9tqyBx1oWEL6TA78a8wcLCUJ%2Bc%2FqIUH0IlyHt5QDMTPyMPeikLkGOr8BeiUz9sSMhabxPS8RJfUCfe2uFNR%2BVSG%2Bze9t%2B%2F%2BYql9BDHJIHwbRev7aD1%2FA0uOwSuVCv9zMoGYkevIvvV6..
